## Deliverable 2. Create a Customer Travel Destinations Map.

In [10]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Keats,AU,-14.2500,129.5500,88.92,53,3,7.74,clear sky
1,1,Tuatapere,NZ,-46.1333,167.6833,53.82,65,90,10.42,overcast clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,42.46,56,20,18.41,few clouds
3,3,Maun,BW,-19.9833,23.4167,80.56,36,0,5.75,clear sky
4,4,Kuytun,RU,54.3419,101.5036,34.63,71,83,6.93,broken clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Keats,AU,-14.2500,129.5500,88.92,53,3,7.74,clear sky
3,3,Maun,BW,-19.9833,23.4167,80.56,36,0,5.75,clear sky
5,5,Faanui,PF,-16.4833,-151.7500,79.18,73,0,17.60,clear sky
14,14,Saint Anthony,US,45.0205,-93.2180,75.70,77,90,3.44,overcast clouds
18,18,Vaini,TO,-21.2000,-175.2000,75.36,88,75,9.22,broken clouds
21,21,Poum,NC,-20.2333,164.0167,75.51,81,62,24.58,broken clouds
25,25,Emerald,AU,-23.5333,148.1667,80.98,36,40,12.26,scattered clouds
26,26,Hithadhoo,MV,-0.6000,73.0833,81.39,74,100,3.49,overcast clouds
29,29,Hilo,US,19.7297,-155.0900,81.28,92,90,3.44,moderate rain
30,30,Aransas Pass,US,27.9095,-97.1500,82.40,85,1,7.00,clear sky


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# There is no empty code, thus no rows need to be dropped. 
clean_df = preferred_cities_df

In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Port Keats,AU,88.92,clear sky,-14.2500,129.5500,
3,Maun,BW,80.56,clear sky,-19.9833,23.4167,
5,Faanui,PF,79.18,clear sky,-16.4833,-151.7500,
14,Saint Anthony,US,75.70,overcast clouds,45.0205,-93.2180,
18,Vaini,TO,75.36,broken clouds,-21.2000,-175.2000,
21,Poum,NC,75.51,broken clouds,-20.2333,164.0167,
25,Emerald,AU,80.98,scattered clouds,-23.5333,148.1667,
26,Hithadhoo,MV,81.39,overcast clouds,-0.6000,73.0833,
29,Hilo,US,81.28,moderate rain,19.7297,-155.0900,
30,Aransas Pass,US,82.40,clear sky,27.9095,-97.1500,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Port Keats,AU,88.92,clear sky,-14.2500,129.5500,TDC Village (Hundred Man Camp)
3,Maun,BW,80.56,clear sky,-19.9833,23.4167,Enviro Villa
5,Faanui,PF,79.18,clear sky,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
14,Saint Anthony,US,75.70,overcast clouds,45.0205,-93.2180,"Holiday Inn Express Roseville-St. Paul, an IHG..."
18,Vaini,TO,75.36,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
...,...,...,...,...,...,...,...
680,Pitimbu,BR,75.79,few clouds,-7.4706,-34.8086,Reserva do Abiaí
681,Mana,GF,79.41,scattered clouds,5.6592,-53.7767,LE SAMANA
683,Samut Songkhram,TH,82.94,overcast clouds,13.4000,99.9583,Amphawa Nanon Hotel & Spa
690,Tecolutla,MX,82.40,overcast clouds,20.4833,-97.0167,Hotel Plaza Marsol


In [22]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacations.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))